In [1]:
import pandas as pd
import re
from rapidfuzz import fuzz, process

# === Unit and fraction setup ===
unit_variants = [unit.lower() for unit in [
    "cup", "cups", "gram", "grams", "g", "g.", "gm", "gm.", "kg",
    "ml", "ltr", "ltr.", "lt", "lt.", "litre", "litres", "pound",
    "no", "no.", "nos", "nos.", "pc", "pc.", "pcs", "pcs.", "millilitre", "millilitres",
    "tsp", "tsp.", "tbsp", "tbsp.", "spoon", "spoons", "spoonful", "scoop",
    "tablespoon", "tablespoons", "table spoon", "table spoons", "bottle", "bottles",
    "pinch", "piece", "pieces", "inch", "inches", "packet", "packets", "bunch", "glass",
    "pkt", "pt", "pk", "teaspoon", "teaspoons", "tea spoon", "tea spoons", "slice", "slices", "sprig",
    "kilo", "liter", "liters", "oz", "oz.", "a pinch", "handful", "handfull", "hand full", "hand ful",
    "medium-size", "medium size", "medium-sized", "medium sized", "sachet", "sachets", "bowl"
]]

unicode_fraction_map = {
    '½': '1/2', '¼': '1/4', '¾': '3/4',
    '⅓': '1/3', '⅔': '2/3', '⅛': '1/8',
    '⅜': '3/8', '⅝': '5/8', '⅞': '7/8'
}

def normalize_fractions(text):
    for uni_frac, ascii_frac in unicode_fraction_map.items():
        text = text.replace(uni_frac, f' {ascii_frac}')
    return text

def extract_parts(line):
    line = str(line).strip()
    line = normalize_fractions(line)
    line = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', line)
    line = line.replace('&', ' ')
    line = re.sub(r'^\s*([a-zA-Z0-9]+[.)-])\s*', '', line)
    line = line.replace(',', ' ')  # treat comma as space

    quantity, unit, ingredient = '', '', ''
    words = line.split()
    if not words:
        return quantity, unit, ingredient

    # Handle "cardamom powder 2 tbsp"
    if len(words) >= 2:
        last = words[-1].lower()
        second_last = words[-2].lower()

        if re.match(r'^(\d*\.\d+|\d+/\d+|\d+[-–]\d+|\d+)$', second_last):
            match, score, _ = process.extractOne(last, unit_variants, scorer=fuzz.token_sort_ratio)
            if score >= 80:
                quantity, unit = second_last, last
                ingredient = ' '.join(words[:-2])
                return quantity.strip(), unit.strip(), ingredient.strip()

        match, score, _ = process.extractOne(second_last, unit_variants, scorer=fuzz.token_sort_ratio)
        if re.match(r'^(\d*\.\d+|\d+/\d+|\d+[-–]?\d*)$', last) and score >= 80:
            quantity, unit = last, second_last
            ingredient = ' '.join(words[:-2])
            return quantity.strip(), unit.strip(), ingredient.strip()

    # Handle "2 tbsp cardamom powder"
    if len(words) >= 2 and re.match(r'^\d+$', words[0]) and re.match(r'^\d+$', words[1]):
        words[0] = f"{words[0]}-{words[1]}"
        words.pop(1)

    if len(words) >= 2 and re.match(r'^\d+$', words[0]) and re.match(r'^\d+/\d+$', words[1]):
        quantity = f"{words[0]} {words[1]}"
        words = words[2:]
    elif re.match(r'^(\d*\.\d+|\d+/\d+|\d+[-–]?\d*)$', words[0]):
        quantity = words[0]
        words = words[1:]
    elif re.match(r'^\d+$', words[0]):
        quantity = words[0]
        words = words[1:]

    if words:
        match, score, _ = process.extractOne(words[0].lower(), unit_variants, scorer=fuzz.token_sort_ratio)
        if score >= 80:
            unit = words[0]
            words = words[1:]

    ingredient = ' '.join(words)
    return quantity.strip(), unit.strip(), ingredient.strip()

# === Load input ===
df = pd.read_excel("ingredient_column.xlsx")

output_rows = []

# Assign group_id to preserve blank-line separation
group_counter = 0
previous_video_id = None
previous_blank = True

for _, row in df.iterrows():
    video_id = str(row.get("video_id", "")).strip()
    cell = str(row.get("ingredients", "")).strip()

    if not cell:
        output_rows.append({
            "video_id": video_id,
            "group_id": group_counter,
            "original_line": "",
            "quantity": "",
            "unit": "",
            "ingredient": ""
        })
        previous_blank = True
        continue

    lines = cell.split('\n')
    has_content = False

    for line in lines:
        line = line.strip()
        if not line:
            continue  # ignore blank lines inside the cell

        # New group if blank row before or new video
        if previous_blank or video_id != previous_video_id:
            group_counter += 1

        qty, unit, ingr = extract_parts(line)

        output_rows.append({
            "video_id": video_id,
            "group_id": group_counter,
            "original_line": line,
            "quantity": qty,
            "unit": unit,
            "ingredient": ingr
        })

        has_content = True
        previous_blank = False
        previous_video_id = video_id

    # After all lines from one original row are processed, add a blank separator
    if has_content:
        output_rows.append({
            "video_id": video_id,
            "group_id": group_counter,
            "original_line": "",
            "quantity": "",
            "unit": "",
            "ingredient": ""
        })
        previous_blank = True


# === Export exploded output ===
exploded_df = pd.DataFrame(output_rows)
exploded_df.to_excel("exploded_output_by_video_id.xlsx", index=False)
print(" Exploded file saved: exploded_output_by_video_id.xlsx")


✅ Exploded file saved: exploded_output_by_video_id.xlsx
